In [6]:

import psycopg2 as pg2
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import pickle
from datetime import datetime, date, timedelta
import importlib
from  BidTrackerScraper import BidTrackerScraper
from bs4 import BeautifulSoup
import pymongo



In [2]:
print ("Connecting to SQL")
conn = pg2.connect(user='postgres',  dbname='penny', host='localhost', port='5432', password='password')

print ("Reading Dataset")
df = pd.read_sql ("Select username, count(*) from bids group by username order by count(*) desc", conn)


Connecting to SQL
Reading Dataset


In [7]:
myclient = pymongo.MongoClient("mongodb://localhost:27017/")
db = myclient["penny"]
pages_collection = db["pages"]


In [4]:
auction_ids

NameError: name 'auction_ids' is not defined

In [8]:
bts = BidTrackerScraper()
bts.login(3)

200
200


In [9]:
titles = {}
for username in df.username:
    print (username)
    url = "http://www.bidtracker.info/UserAuctions?site=quibids&user="+username
    r = bts._session.get(url)
    soup = BeautifulSoup(r.text, 'html.parser')
    rows = soup.findAll("tr", {"class": {"tr-winner","tr-ended"}})
    for r in rows:
        tds = r.findAll("td", {"class": "td-inner"})
        auction_id = tds[0].text
        title = tds[1].text
        if not ("Gift Card" in title or "Buster" in title or "Voucher" in title):
            continue
        if "$10 " in title:
            continue
        if "$15 " in title:
            continue
#        if "$75 Amazon Gift Card AND 10 Bids" in title:
#            continue
#        if "$100 Amazon Gift Card AND 20 Bids" in title:
#            continue
#        if "$75 Amazon Gift Card AND 20 Bids" in title:
#            continue
        cashvalue = tds[2].text
        datestr = tds[8].text
        datestr = datestr.split(" ")[0]
        if (len(datestr)==6):
            datestr += "-20"
            
        date = datetime.strptime(datestr, "%b-%d-%y")
        if (datetime.now()-date).days > 1 and date >= datetime(2020, 1, 1):
            #print (auction_id, int(auction_id) in set(auction_ids.qauctionid), date, title)
            item_found = pages_collection.find({"_id": auction_id},{"_id": 1}).count() > 0
            if (not item_found):
               # print(auction_id,title, datestr)

                if "%" in title:
                    title = title.split("%")[1]
                if title.startswith("L"):
                    title = title[1:]
                if (title not in titles.keys()):
                    titles[title] = set()
                titles[title].add(auction_id)
    for t in titles.keys():
        print(t, len(titles[t]))
    

mlee1168


/home/ubuntu/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:33: DeprecationWarning: count is deprecated. Use Collection.count_documents instead.


BidsWins2016
fixit08
jcloeb
UQuitnow
WINITIORBUYIT
debby1228
LMSretired48
DreamOn4U
webuynow
youneverknow05
RayRice
cindyboogs47
Jitterrox
Missy44tym
koolsurfer19
reddragon09
dstehlin49
HIGHBIDDER1956
dnkwalker
kargarks
vivace
1TAYCAN
DGDG26
07robert


KeyboardInterrupt: 

In [156]:
new_auction_collection = db["new_auctions"]


In [165]:
titles.keys()

dict_keys(['25 Voucher Bids', '50 Voucher Bids', '15 Voucher Bids', '$25 Target Gift Card AND 10 Bids', '$25 Amazon Gift Card', '$25 Walmart Gift Card', '_id'])

In [166]:
titles_new = {}
for t in titles.keys():
    if not t.startswith('$'):
        titles_new[t]= titles[t]

In [70]:
t

True

In [73]:
155435135 in pd.Series(auction_ids.qauctionid)

False

In [83]:
155435135 in set(auction_ids.qauctionid)

True

In [16]:
new_auction_collection = db["new_auctions"]
for t in new_auction_collection.find({}):
    print(t['50 Voucher Bids'])

['280073632', '816029407', '281624386', '532555505', '729705541', '510850984', '785038512', '618544564', '482537169', '704365170', '866754756', '907182460', '612097750', '546698789', '243657046', '723901974', '895664397', '451884391', '546229160', '515956025', '856665143', '146891600', '340051294', '455503395', '794395613', '624932993', '823284585', '165437863', '825130191', '377893318', '701653648', '282365369', '862044816', '309976193', '177883123', '673464514', '612057664', '268298792', '471624997', '607288616', '144440240', '244083179', '613154261', '615775975', '973494096', '198051426', '678925872', '630254608', '114505735', '201604242', '471508519', '158484091', '443609631', '694894263', '301680238', '622554365', '761090005', '336449553', '959963730', '276399821', '917375198', '614865986', '153691069', '794152239', '632605897', '603870367', '725550879', '913010519', '575907015', '777069282', '996370876', '288198585', '680784568', '287435535', '877668729', '979243375', '162696187'

In [17]:
conn.close()

NameError: name 'conn' is not defined